In [ ]:
#%% ライブラリのインポート
import json
import sys
from argparse import Namespace
from pathlib import Path
from time import time

import numpy as np
import torch
import resnet3d

# sys.path.append("resnet3d/")
import torchvision
from IPython.display import HTML
from resnet3d.main import generate_model, get_inference_utils, resume_model
from resnet3d.model import generate_model, make_data_parallel
from torch.backends import cudnn
from torchvision.transforms import transforms
from torchvision.transforms.transforms import Normalize, ToPILImage

sys.path.append("src/")

from utils.dataset_utils import load_jpg_ucf101_from_NAS
from utils.sensitivity_analysis import OcclusionSensitivityMap3D as OSM
from utils.visualization import visualize

In [ ]:
#%% モデルの準備〜入力の準備
opt_path = "data/r3d_models/finetuning/ucf101/r3d50_K_fc/opts.json"
with open(opt_path, "r") as f:
    model_opt = json.load(f)
model_opt = Namespace(**model_opt)

model_opt.device = torch.device("cpu" if model_opt.no_cuda else "cuda")
if not model_opt.no_cuda:
    cudnn.benchmark = True
if model_opt.accimage:
    torchvision.set_image_backend("accimage")

model_opt.ngpus_per_node = torch.cuda.device_count()

model = generate_model(model_opt)
model = resume_model(model_opt.resume_path, model_opt.arch, model)
model = make_data_parallel(model, model_opt.distributed, model_opt.device)
model.eval()

model_opt.inference_batch_size = 1
for attribute in dir(model_opt):
    if "path" in str(attribute) and getattr(model_opt, str(attribute)) != None:
        setattr(model_opt, str(attribute), Path(getattr(model_opt, str(attribute))))
inference_loader, inference_class_names = get_inference_utils(model_opt)

class_labels_map = {v.lower(): k for k, v in inference_class_names.items()}


loading checkpoint data/r3d_models/finetuning/ucf101/r3d50_K_fc/save_200.pth model
model: ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [ ]:
#%%
inputs, targets = iter(inference_loader).__next__()
video_size = inputs[[0]].shape
transform = inference_loader.dataset.spatial_transform

_transforms = transform.transforms
idx = [type(i) for i in _transforms].index(resnet3d.spatial_transforms.Normalize)
normalize = _transforms[idx]
mean = torch.tensor(normalize.mean)
std = torch.tensor(normalize.std)

unnormalize = transforms.Compose(
    [
        Normalize((-mean / std).tolist(), (1 / std).tolist()),
        ToPILImage(),
    ]
)


In [ ]:
#%% OSMインスタンスの作成
spatial_crop_size = 16
spatial_stride = 8
temporal_stride = 2

aosa_single = OSM(
    net=model,
    video_size=video_size,
    device=model_opt.device,
    spatial_crop_size=spatial_crop_size,
    spatial_stride=spatial_stride,
    temporal_stride=temporal_stride,
    transform=transform,
    batchsize=400,
    N_stack_mask=1,
)

aosa = OSM(
    net=model,
    video_size=video_size,
    device=model_opt.device,
    spatial_crop_size=spatial_crop_size,
    spatial_stride=spatial_stride,
    temporal_stride=temporal_stride,
    transform=transform,
    batchsize=400,
    N_stack_mask=8,
)

arsa = []
max_masks = 12

for i in range(max_masks):
    arsa.append(
        OSM(
            net=model,
            video_size=video_size,
            device=model_opt.device,
            spatial_crop_size=spatial_crop_size,
            spatial_stride=spatial_stride,
            temporal_stride=temporal_stride,
            transform=transform,
            batchsize=400,
            N_stack_mask=i+1,
        )
    )


In [ ]:
import re

# ファイルのパス
file_path = '/mnt/data/users/nagatomo/ucfTrainTestlist/testlist01.txt'

# 正規表現パターン：最初のクラス名、gの後の数字、cの後の番号を抽出
pattern = r'^([^/]+).*?g(\d+)_c(\d+)'

# 結果を格納するリスト
class_to_number = {}
matches = []
count = 1

# ファイルを開いて最初のクラス名に番号を付ける
with open(file_path, 'r') as file:
    for line in file:
        # 最初のクラス名を抽出
        match = re.match(pattern, line)
        if match:
            class_name = match.group(1)  # クラス名
            # クラス名がまだ登録されていなければ、番号を付けて保存
            if class_name not in class_to_number:
                class_to_number[class_name] = count
                count += 1

# ファイルを再度読み込んで番号を付けてgとcの数字を抽出
with open(file_path, 'r') as file:
    for line in file:
        match = re.match(pattern, line)
        if match:
            class_name = match.group(1)  # クラス名
            group_number = match.group(2)  # gの後の数字
            count_number = match.group(3)  # cの後の番号
            # クラス名を番号に置き換え
            class_number = class_to_number[class_name]
            matches.append((int(class_number)-1, int(group_number), int(count_number)))

# 結果を表示
# for class_number, group_number, count_number in matches:
#     print(f"Class: {class_number}, Group: {group_number}, Count: {count_number}")


In [ ]:
from evaluation.evaluation import CausalMetric, auc, gkern
import torch.nn as nn
from tqdm import tqdm

klen = 11
ksig = 5
kern = gkern(klen, ksig)

def blur_video(x):
    out = x.copy()
    for i in range(16):
        out[0, i] = nn.functional.conv2d(x[0, i], kern, padding=klen//2)

# Function that blurs input image
blur = lambda x: nn.functional.conv2d(x, kern, padding=klen//2)

In [ ]:
insertion = CausalMetric(model, 'ins', 112, substrate_fn=blur)
deletion = CausalMetric(model, 'del', 112, substrate_fn=torch.zeros_like)
# torch.zeros_like は，ある tensor の値をすべて0にしたいときに使う．

In [ ]:
#%% AOSAの実行

# l = 20 # 正解クラス
# g = 1  # > 0
# c = 1  # > 0
n = 0  # 動画の前からどれくらいか1~5くらいがいいのではないかと思われる
        # 0じゃないとだめなやつあった．
scores_aosa_single = {'del': [], 'ins': []}
scores_aosa = {'del': [], 'ins': []}

cnt = 0

for l, g, c in tqdm(matches):
    video = load_jpg_ucf101_from_NAS(l, g, c, n, inference_class_names, transform)
    if video is False:
        continue
    video = video.transpose(0, 1)
    target = l
    with torch.inference_mode():
        pred = model(video.unsqueeze(0)).cpu().numpy().argmax()
    video_orgimg = []

    for i in range(video_size[2]):
        img = video.squeeze().transpose(0, 1)[i]
        video_orgimg.append(np.array(unnormalize(img)))
    video_orgimg = np.array(video_orgimg)

    aosa_single_map = aosa_single.run(video, target)

    aosa_map = aosa.run(video, target)

    # Evaluate deletion
    h = deletion.evaluate_video(pred, video, aosa_single_map, verbose=1)
    scores_aosa_single['del'].append(auc(h))

    h = deletion.evaluate_video(pred, video, aosa_map, verbose=1)
    scores_aosa['del'].append(auc(h))

    # Evaluate insertion
    h = insertion.evaluate_video(pred, video, aosa_single_map, verbose=1)
    scores_aosa_single['ins'].append(auc(h))

    h = insertion.evaluate_video(pred, video, aosa_map, verbose=1)
    scores_aosa['ins'].append(auc(h))
    cnt += 1
    if cnt % 200 == 0:
        print('{} / 3783:\nAOSA Single Deletion - {:.5f}\nAOSA Single Insertion - {:.5f}'.format(cnt, np.mean(scores_aosa_single['del']), np.mean(scores_aosa_single['ins'])))
        print('{} / 3783:\nAOSA Deletion - {:.5f}\nAOSA Insertion - {:.5f}'.format(cnt, np.mean(scores_aosa['del']), np.mean(scores_aosa['ins'])))

    
print('----------------------------------------------------------------')
print('Final:\nAOSA Single Deletion - {:.5f}\nAOSA Single Insertion - {:.5f}'.format(np.mean(scores_aosa_single['del']), np.mean(scores_aosa_single['ins'])))
print('Final:\nAOSA Deletion - {:.5f}\nAOSA Insertion - {:.5f}'.format(np.mean(scores_aosa['del']), np.mean(scores_aosa['ins'])))


  5%|▌         | 200/3783 [23:20<6:34:16,  6.60s/it]

200 / 3783:
AOSA Single Deletion - 0.12175
AOSA Single Insertion - 0.67683
200 / 3783:
AOSA Deletion - 0.10751
AOSA Insertion - 0.67755


 11%|█         | 400/3783 [45:55<6:27:21,  6.87s/it]

400 / 3783:
AOSA Single Deletion - 0.17031
AOSA Single Insertion - 0.67825
400 / 3783:
AOSA Deletion - 0.15151
AOSA Insertion - 0.68381


 16%|█▌        | 600/3783 [1:08:25<5:57:14,  6.73s/it]

...

3400 / 3783:
AOSA Single Deletion - 0.14550
AOSA Single Insertion - 0.67780
3400 / 3783:
AOSA Deletion - 0.13685
AOSA Insertion - 0.66957


 96%|█████████▌| 3628/3783 [6:48:46<16:58,  6.57s/it]

3600 / 3783:
AOSA Single Deletion - 0.14508
AOSA Single Insertion - 0.67631
3600 / 3783:
AOSA Deletion - 0.13665
AOSA Insertion - 0.66782


100%|██████████| 3783/3783 [7:06:02<00:00,  6.76s/it]

----------------------------------------------------------------
Final:
AOSA Single Deletion - 0.14731
AOSA Single Insertion - 0.67439
Final:
AOSA Deletion - 0.13831
AOSA Insertion - 0.66589


In [ ]:
#%% ARSAの実行

# l = 20 # 正解クラス
# g = 1  # > 0
# c = 1  # > 0
n = 0  # 動画の前からどれくらいか1~5くらいがいいのではないかと思われる
        # 0じゃないとだめなやつあった．
scores_arsa_single = {'del': [], 'ins': []}
scores_arsa = {'del': [], 'ins': []}

cnt = 0
print(f"mask: {arsa[10].N_stack_mask}, {arsa[11].N_stack_mask}")

for l, g, c in tqdm(matches):
    video = load_jpg_ucf101_from_NAS(l, g, c, n, inference_class_names, transform)
    if video is False:
        continue
    video = video.transpose(0, 1)
    target = l
    with torch.inference_mode():
        pred = model(video.unsqueeze(0)).cpu().numpy().argmax()
    video_orgimg = []

    for i in range(video_size[2]):
        img = video.squeeze().transpose(0, 1)[i]
        video_orgimg.append(np.array(unnormalize(img)))
    video_orgimg = np.array(video_orgimg)

    arsa_single_map = arsa[10].inverse_run(video, target)

    arsa_map = arsa[11].inverse_run(video, target)

    # Evaluate deletion
    h = deletion.evaluate_video(pred, video, arsa_single_map, verbose=1)
    scores_arsa_single['del'].append(auc(h))

    h = deletion.evaluate_video(pred, video, arsa_map, verbose=1)
    scores_arsa['del'].append(auc(h))

    # Evaluate insertion
    h = insertion.evaluate_video(pred, video, arsa_single_map, verbose=1)
    scores_arsa_single['ins'].append(auc(h))

    h = insertion.evaluate_video(pred, video, arsa_map, verbose=1)
    scores_arsa['ins'].append(auc(h))
    cnt += 1
    if cnt % 200 == 0:
        print('{} / 3783:\nARSA Single Deletion - {:.5f}\nARSA Single Insertion - {:.5f}'.format(cnt, np.mean(scores_arsa_single['del']), np.mean(scores_arsa_single['ins'])))
        print('{} / 3783:\nARSA Deletion - {:.5f}\nARSA Insertion - {:.5f}'.format(cnt, np.mean(scores_arsa['del']), np.mean(scores_arsa['ins'])))

    
print('----------------------------------------------------------------')
print('Final:\nARSA Single Deletion - {:.5f}\nARSA Single Insertion - {:.5f}'.format(np.mean(scores_arsa_single['del']), np.mean(scores_arsa_single['ins'])))
print('Final:\nARSA Deletion - {:.5f}\nARSA Insertion - {:.5f}'.format(np.mean(scores_arsa['del']), np.mean(scores_arsa['ins'])))


mask: 11, 12


  5%|▌         | 200/3783 [24:22<7:03:51,  7.10s/it]

200 / 3783:
ARSA Single Deletion - 0.12714
ARSA Single Insertion - 0.65047
200 / 3783:
ARSA Deletion - 0.12947
ARSA Insertion - 0.65034


 11%|█         | 400/3783 [49:19<6:53:50,  7.34s/it]

400 / 3783:
ARSA Single Deletion - 0.17004
ARSA Single Insertion - 0.67076
400 / 3783:
ARSA Deletion - 0.17136
ARSA Insertion - 0.66991


...

 96%|█████████▌| 3628/3783 [7:16:50<18:07,  7.02s/it]

3600 / 3783:
ARSA Single Deletion - 0.15732
ARSA Single Insertion - 0.65420
3600 / 3783:
ARSA Deletion - 0.15802
ARSA Insertion - 0.65425


100%|██████████| 3783/3783 [7:35:20<00:00,  7.22s/it]

----------------------------------------------------------------
Final:
ARSA Single Deletion - 0.15892
ARSA Single Insertion - 0.65234
Final:
ARSA Deletion - 0.15955
ARSA Insertion - 0.65235


: 